## Data Analysis

In [2]:
#Imports
import pandas as pd
import matplotlib as plot

### Meeting Room w/ Blinds

In [3]:
#Load csv files
blinds_trial = pd.read_csv('power_profiler_scan/pp_meeting_floor3_blinds_scan.csv')

#Only record data between times where actually scanning with program, drop unnecessary columns, 
#and normalize input for time for analysis 
drop_columns = ['time', 'events']
blinds_trial.drop(index=0, inplace=True)

blinds_trial = blinds_trial.apply(pd.to_numeric, errors='coerce')
blinds_0 = blinds_trial[(blinds_trial['time'] >= 58) & (blinds_trial['time'] <= 234)]
blinds = blinds_0.drop(columns=drop_columns)
avg_blinds_0 = blinds_0.mean(numeric_only=True)

with open('blinds.txt', 'w') as file: 
    file.write("Average output for Meeting Room (Blinds) while scanning:\n")
    file.write(f"{avg_blinds_0}\n")

### Meeting Room w/ Windows

In [6]:
#Load csv files 
windows_trial = pd.read_csv('power_profiler_scan/pp_meeting_floor3_windows_scan.csv')

#Only record data between times where actually scanning with program, drop unnecessary columns, 
#and normalize input for time for analysis 
drop_columns = ['time', 'events']
windows_trial.drop(index=0, inplace=True)

windows_trial = windows_trial.apply(pd.to_numeric, errors='coerce')

windows_0 = windows_trial[(windows_trial['time'] >= 46) & (windows_trial['time'] <= 194)]
windows_0 = windows_0.drop(columns=drop_columns)
avg_windows_0 = windows_0.mean(numeric_only=True)
med_windows_0 = windows_0.median(numeric_only=True)
sd_windows_0 = windows_0.std(numeric_only=True)

with open('windows.txt', 'w') as file:
    file.write("Output for Meeting Room (Windows) while scanning:\n")
    file.write("Mean\n")
    file.write(f"{avg_windows_0}\n")
    file.write("\n\nMedian\n")
    file.write(f"{med_windows_0}\n")
    file.write("\n\nStandard Deviation\n")
    file.write(f"{sd_windows_0}\n")
    